In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

**Libraries** 

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
import os
import string
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import math
import re
from sklearn.model_selection import train_test_split
import collections
from keras.models import Sequential
from keras.layers import CuDNNGRU, Dense, Bidirectional, Dropout,MaxPool1D,Conv1D
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.models import load_model

**Basic Cleaning***


In [ ]:
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")
def clean_text(text):
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)

    ## Remove puncuation
#     text = text.translate(string.punctuation)

    ## Convert words to lower case and split them
    text = text.lower()

    ## Remove stop words
#     stops = set(stopwords.words("english"))
#     text = [w for w in text if not w in stops and len(w) >= 3]

#     text = " ".join(text)
    ## Clean the text
    
    return text
train_df['clean_text'] = train_df['question_text'].map(clean_text)
test_df['clean_text'] = test_df['question_text'].map(clean_text)



In [ ]:
#create a vocabulary and filter with 80k top features
vocabulary_dict ={}
all_train_list = train_df['clean_text'].tolist()
for line in all_train_list:
    words = line.split()
    for w in words:
        if w not in vocabulary_dict.keys():
            vocabulary_dict[w] = 1
        else:
            vocabulary_dict[w] = vocabulary_dict[w] +1
vocabulary_dict= collections.Counter(vocabulary_dict)
lst = vocabulary_dict.most_common(80000)
df = pd.DataFrame(lst, columns = ['Word', 'Count'])

In [ ]:
df.head()

In [ ]:
#divide into train and validation
train_df, val_df = train_test_split(train_df, test_size=0.1)
train_df.head()

**Embedding Setups**

In [ ]:
# embdedding setup we have taken glove embeddings for creating word2array dicts

embeddings_index = {}
f = open('../input/embeddings/glove.840B.300d/glove.840B.300d.txt')
for line in tqdm(f):
    values = line.split(" ")
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
# Convert values to embeddings
def text_to_array(text):
    empyt_emb = np.zeros(300)
    text = text[:-1].split()[:30]
    embeds = [embeddings_index.get(x, empyt_emb) for x in text]
    embeds+= [empyt_emb] * (30 - len(embeds))
    return np.array(embeds)
# train_vects = [text_to_array(X_text) for X_text in tqdm(train_df["question_text"])]
val_vects = np.array([text_to_array(X_text) for X_text in tqdm(val_df["clean_text"][:10000])])
val_y = np.array(val_df["target"][:10000])

In [ ]:
# Data providers
batch_size = 128

def batch_gen(train_df):
    n_batches = math.ceil(len(train_df) / batch_size)
    while True:
        train_df = train_df.sample(frac=1.)  # Shuffle the data.
        for i in range(n_batches):
            texts = train_df.iloc[i*batch_size:(i+1)*batch_size, 1]
            text_arr = np.array([text_to_array(text) for text in texts])
            yield text_arr, np.array(train_df["target"][i*batch_size:(i+1)*batch_size])

In [ ]:
train_df = train_df[['qid','clean_text','target']]
train_df.head()

In [ ]:
train_gen = batch_gen(train_df)

**Modellling Setup**

In [ ]:
model = Sequential()

model.add(Bidirectional(CuDNNGRU(256, return_sequences=True),
                       input_shape=(30, 300)))
model.add(Conv1D(128,5))
model.add(MaxPool1D(5))
model.add(Dropout(0.2))
model.add(Bidirectional(CuDNNGRU(256)))
model.add(Dropout(0.2))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:

model.fit_generator(train_gen, epochs=20,
                    steps_per_epoch=1000,
                    validation_data=(val_vects, val_y),
                    verbose=True,callbacks=callbacks_list)

**Prediction Made on Test Data**

In [ ]:
ls

In [ ]:
model_ = load_model('weights-improvement-17-0.96.hdf5')

In [ ]:
def batch_gen_test(test_df):
    n_batches = math.ceil(len(test_df) / 256)
    for i in range(n_batches):
        texts = test_df.iloc[i*batch_size:(i+1)*batch_size, 1]
        text_arr = np.array([text_to_array(text) for text in texts])
        yield text_arr

test_df = test_df[['qid','clean_text']]
# test_gen = batch_gen_test(test_df)
    

In [ ]:
def return_predictons(x):
    pred =model_.predict(text_to_array(x).reshape(1,30,300))[0][0]
    return int(np.round(pred))
test_df['predictions'] = test_df['clean_text'].map(return_predictons)
test_df.head()

In [ ]:
submission_df = test_df[['qid','predictions']]
submission_df.rename(columns={'predictions':'prediction'},inplace=True)

In [ ]:
submission_df.head()

In [ ]:
ls

In [ ]:
submission_df.to_csv('submission.csv',index=False)

In [ ]:
ls